In [5]:
pip install beyond

Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
from beyond.io.tle import Tle
from beyond.frames import create_station
from beyond.dates import Date, timedelta

# Parse TLE
tle = Tle("""NOAA 15 [B]             
1 25338U 98030A   20205.85092036  .00000065  00000-0  45627-4 0  9992
2 25338  98.7149 230.7131 0010993 151.5401 208.6381 14.25980908154168""")

# Create a station from which to compute the pass
station = create_station('Norwich', (52, 1, 0))

counter = 0

for orb in station.visibility(tle.orbit(), start=Date.now(), stop=timedelta(days=2), step=timedelta(minutes=2), events=True):

    # As all angles are given in radians,
    # there is some conversion to do
    azim = -np.degrees(orb.theta) % 360
    elev = np.degrees(orb.phi)
    r = orb.r / 1000.

    print("{event:10} {tle.name}  {date:%Y-%m-%dT%H:%M:%S.%f} {azim:7.2f} {elev:7.2f} {r:10.2f}".format(
        date=orb.date, r=r, azim=azim, elev=elev,
        tle=tle, event=orb.event if orb.event is not None else ""
    ))

    # Stop at the end of the first pass
    if orb.event and orb.event.info == "LOS":
        counter += 1
        if counter >= 5:
            break




AOS 0 Norwich NOAA 15 [B]  2025-07-21T19:13:09.770135  157.02    0.00    3318.31
           NOAA 15 [B]  2025-07-21T19:13:21.720597  156.96    0.73    3238.56
           NOAA 15 [B]  2025-07-21T19:15:21.720597  155.97    9.40    2442.08
           NOAA 15 [B]  2025-07-21T19:17:21.720597  153.58   22.78    1675.31
           NOAA 15 [B]  2025-07-21T19:19:21.720597  144.03   49.69    1032.70
MAX Norwich NOAA 15 [B]  2025-07-21T19:20:46.536155   71.34   76.46     840.59
           NOAA 15 [B]  2025-07-21T19:21:21.720597   18.02   67.89     877.42
           NOAA 15 [B]  2025-07-21T19:23:21.720597  351.36   31.52    1385.82
           NOAA 15 [B]  2025-07-21T19:25:21.720597  347.61   14.31    2121.10
           NOAA 15 [B]  2025-07-21T19:27:21.720597  346.36    4.23    2909.24
LOS 0 Norwich NOAA 15 [B]  2025-07-21T19:28:27.680103  346.05   -0.00    3348.27
AOS 0 Norwich NOAA 15 [B]  2025-07-21T20:54:21.419882  207.72    0.00    3318.99
           NOAA 15 [B]  2025-07-21T20:55:21.720597  21

In [7]:
import matplotlib.pyplot as plt

In [8]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pathlib import Path

from beyond.io.tle import Tle
from beyond.dates import Date, timedelta


# Parsing of TLE
tle = Tle("""NOAA 15 [B]             
1 25338U 98030A   20191.81750536  .00000057  00000-0  42639-4 0  9999
2 25338  98.7153 216.7534 0010156 191.4103 168.6848 14.25977849152168""")

# Create a station from which to compute the pass
# Conversion into `Orbit` object
orb = tle.orbit()

# Tables containing the positions of the ground track
latitudes, longitudes = [], []
prev_lon, prev_lat = None, None

period = orb.infos.period

#start = orb.date - period
start = Date.now()
stop = 1 * period
step = period / 100

for point in orb.ephemeris(start=start, stop=stop, step=step):

    # Conversion to earth rotating frame
    point.frame = 'ITRF'

    # Conversion from cartesian to spherical coordinates (range, latitude, longitude)
    point.form = 'spherical'

    # Conversion from radians to degrees
    lon, lat = np.degrees(point[1:3])

    # Creation of multiple segments in order to not have a ground track
    # doing impossible paths
    if prev_lon is None:
        lons = []
        lats = []
        longitudes.append(lons)
        latitudes.append(lats)
    elif orb.i < np.pi /2 and (np.sign(prev_lon) == 1 and np.sign(lon) == -1):
        lons.append(lon + 360)
        lats.append(lat)
        lons = [prev_lon - 360]
        lats = [prev_lat]
        longitudes.append(lons)
        latitudes.append(lats)
    elif orb.i > np.pi/2 and (np.sign(prev_lon) == -1 and np.sign(lon) == 1):
        lons.append(lon - 360)
        lats.append(lat)
        lons = [prev_lon + 360]
        lats = [prev_lat]
        longitudes.append(lons)
        latitudes.append(lats)

    lons.append(lon)
    lats.append(lat)
    prev_lon = lon
    prev_lat = lat

img = "earth2.png"
#with cbook.get_sample_data('earth2.png') as img:
#    im = plt.imread(img)

im = plt.imread(img)
plt.figure(figsize=(15.2, 8.2))
plt.imshow(im, extent=[-180, 180, -90, 90])

for lons, lats in zip(longitudes, latitudes):
    plt.plot(lons, lats, 'w', linestyle=":", linewidth=3)

torb1 = orb.copy(frame='ITRF', form='spherical').propagate(start)
lon, lat = np.degrees(torb1[1:3])
#torb = np.degrees(orb.ephemeris(start=start)[0][1:3])
#lon, lat = np.degrees(orb.copy(frame='ITRF', form='spherical')[1:3])
plt.plot([lon], [lat], 'ro')

#lon, lat = np.degrees(orb.copy(frame='ITRF', form='spherical')[1:3])
#plt.plot([lon], [lat], 'ro')
dir(plt)

#plt.xlim([-20, 20])
#plt.ylim([30, 70])
plt.xlim([-180, 180])
plt.ylim([-90, 90])

plt.grid(True, color='w', linestyle=":", alpha=0.4)
#plt.xticks(range(-20, 20, 2))
#plt.yticks(range(30, 70, 2))
plt.xticks(range(-180, 181, 30))
plt.yticks(range(-90, 91, 30))
plt.tight_layout()

#print(plt.type())
print(dir(plt.axis()))

#plt.axes['bottom'].set_color('#dddddd')
#plt.Axes.spines['top'].set_color('#dddddd') 
#plt.Axes.spines['right'].set_color('red')
#plt.Axes.spines['left'].set_color('red')

#plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'earth2.png'

In [ ]:
torb1 = orb.propagate(start).copy(frame='ITRF', form='spherical')
lon, lat = np.degrees(torb1[1:3])
print("%s %s" % (lon, lat))

lon, lat = np.degrees(orb.copy(frame='ITRF', form='spherical')[1:3])
print("%s %s" % (lon, lat))


In [ ]:
orb.date

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt

from beyond.dates import Date, timedelta
from beyond.io.tle import Tle
from beyond.frames import create_station
from beyond.config import config


tle = Tle("""NOAA 15 [B]             
1 25338U 98030A   20205.85092036  .00000065  00000-0  45627-4 0  9992
2 25338  98.7149 230.7131 0010993 151.5401 208.6381 14.25980908154168""")

# Create a station from which to compute the pass
station = create_station('Norwich', (52.6, 1.19, 0))
azims, elevs = [], []

orbit = tle.orbit()

start = None
end = None
max = max

for orb in station.visibility(orbit, start=Date.now(), stop=timedelta(hours=24), step=timedelta(seconds=30), events=True):
    elev = np.degrees(orb.phi)
    # Radians are counterclockwise and azimuth is clockwise
    azim = np.degrees(-orb.theta) % 360

    # Archive for plotting
    azims.append(azim)
    # Matplotlib actually force 0 to be at the center of the polar plot,
    # so we trick it by inverting the values
    elevs.append(90 - elev)

    r = orb.r / 1000.
    
    if orb.event is not None:
        if orb.event.info =='AOS':
            start = orb.date
        if orb.event.info  == 'MAX':
            max = elev
        
        if orb.event.info  == 'LOS':
            
            end = orb.date
            if max >= 10:
                print("Pass: %s %s %s" % (start, max, end))
    
    #print("{event:7} {orb.date:%H:%M:%S} {azim:7.2f} {elev:7.2f} {r:10.2f} {orb.r_dot:10.2f}".format(
    #    orb=orb, r=r, azim=azim, elev=elev, event=orb.event.info if orb.event is not None else ""
    #))
    
    #print (orb.event.info)

    #if orb.event and orb.event.info.startswith("LOS"):
    #    # We stop at the end of the first pass
    #    print()
    #    break

plt.figure()
ax = plt.subplot(111, projection='polar')
ax.set_theta_direction(-1)
ax.set_theta_zero_location('N')
plt.plot(np.radians(azims), elevs, '.')
ax.set_yticks(range(0, 90, 20))
ax.set_yticklabels(map(str, range(90, 0, -20)))
ax.set_rmax(90)


plt.show()


# Predictions

In [48]:
import requests
import numpy as np
from beyond.dates import Date, timedelta
from beyond.io.tle import Tle
from beyond.frames import create_station
from beyond.config import config


# In[3]:


celestrak_noaa =  "https://celestrak.org/NORAD/elements/gp.php?GROUP=starlink&FORMAT=tle"#https://celestrak.org/NORAD/elements/gp.php?GROUP=geo&FORMAT=tle"

data = requests.get(celestrak_noaa)
data = data.content.decode("utf-8")
rows = data.splitlines()

tles = {}
first = None
second = None
last = None
for row in rows:
    if not first:
        first = row
    elif not second:
        second = row
    else:
        last = row
        name = (first.split("[")[0]).strip()
        tles[name] = """%s
%s
%s""" % (first, second, last)
        first = None
        second= None
        



In [51]:

tles['STARLINK-1115']


'STARLINK-1115           \n1 44973U 20001BM  25202.58334491  .00093621  00000+0  48078-3 0  9998\n2 44973  53.0287 180.0586 0008457   8.2227  63.6300 15.80455118  5929'

In [37]:

tle = Tle(tles['MEXSAT 3'])


In [45]:

station = create_station('mx3', (0.05, -114.8, 0)) # obfuscated slightly


A frame with the name 'mx3' is already registered. Overriding


In [42]:

def predictions(tle, station, hours):
    azims, elevs = [], []

    orbit = tle.orbit()

    start = None
    end = None
    max_alt = None

    for orb in station.visibility(orbit, start=Date.now(), stop=timedelta(hours=hours), step=timedelta(seconds=30), events=True):
        elev = np.degrees(orb.phi)
        # Radians are counterclockwise and azimuth is clockwise
        azim = np.degrees(-orb.theta) % 360

        # Archive for plotting
        azims.append(azim)
        # Matplotlib actually force 0 to be at the center of the polar plot,
        # so we trick it by inverting the values
        elevs.append(90 - elev)

        #r = orb.r / 1000.

        if orb.event is not None:
            if orb.event.info =='AOS': # aquisition of signal
                start = orb.date
            if orb.event.info  == 'MAX': 
                max_alt = elev

            if orb.event.info  == 'LOS': # loss of signal

                end = orb.date
                if max_alt and max_alt >= 10:
                    print("Pass: %s %s %s" % (start, max_alt, end))
                    start = None
                    end = None
                    max_alt = None

In [46]:

predictions(Tle(tles['FLTSATCOM 8 (USA 46)']), station, 24)


In [44]:

predictions(Tle(tles['MEXSAT 3']), station, 24)


In [52]:

predictions(Tle(tles['STARLINK-1132']), station, 24)


Pass: 2025-07-22T02:14:21.771139 UTC 76.58356015168613 2025-07-22T02:27:01.236020 UTC
Pass: 2025-07-22T14:11:08.568727 UTC 83.56232714300702 2025-07-22T14:23:48.609867 UTC


In [53]:
from beyond.config import config
print(config)

{}


In [55]:
pip list

Package                           Version
--------------------------------- ------------------
aiobotocore                       2.12.3
aiohappyeyeballs                  2.4.0
aiohttp                           3.10.5
aioitertools                      0.7.1
aiosignal                         1.2.0
alabaster                         0.7.16
altair                            5.0.1
anaconda-anon-usage               0.4.4
anaconda-catalogs                 0.2.0
anaconda-client                   1.12.3
anaconda-cloud-auth               0.5.1
anaconda-navigator                2.6.3
anaconda-project                  0.11.1
annotated-types                   0.6.0
anyio                             4.2.0
appdirs                           1.4.4
archspec                          0.2.3
argon2-cffi                       21.3.0
argon2-cffi-bindings              21.2.0
arrow                             1.2.3
astroid                           2.14.2
astropy                           6.1.3
astropy-iers-data

In [67]:
from datetime import datetime
from pyorbital.orbital import Orbital

# TLE de ejemplo para la Estación Espacial Internacional (ISS)
# (Estos TLEs pueden variar y deben ser actualizados periódicamente)
tle_line1 = "1 25544U 98067A   24184.62939801  .00008801  00000+0  16254-3 0  9997"
tle_line2 = "2 25544  51.6416 112.9839 0005703  72.0305 288.0838 15.49503940455434"

# Crear un objeto Orbital
# Puedes pasarle un nombre si quieres, o dejarlo en blanco
iss = Orbital("ISS", line1=tle_line1, line2=tle_line2)

# Definir la hora para la que quieres la posición (UTC es lo más común)
# Por ejemplo, ahora mismo (simulado para este ejemplo)
# Asegúrate de que esta fecha y hora sean razonables para el TLE que usas.
time_to_predict = datetime(2025, 7, 3, 12, 0, 0) # 3 de Julio de 2025, 12:00:00 UTC

# Obtener la longitud, latitud y altitud (en grados y kilómetros)
lon, lat, alt = iss.get_lonlatalt(time_to_predict)

print(f"En {time_to_predict} UTC:")
print(f"Longitud: {lon:.4f} grados")
print(f"Latitud: {lat:.4f} grados")
print(f"Altitud: {alt:.4f} km")

ModuleNotFoundError: No module named 'pyorbital'

In [34]:
from datetime import datetime, timezone
from sgp4.api import WGS72, Satrec, jday # Import jday
import numpy as np

# Your provided TLEs
tle_line1 = "1 45047U 20006D   25204.46723892  .00000788  00000-0  71758-4 0  9997"
tle_line2 = "2 45047  53.0542 236.1777 0001106  83.7504 276.3610 15.06403251302242"

# Create a Satrec object from the TLE
satellite = Satrec.twoline2rv(tle_line1, tle_line2)

# Define the time for prediction (UTC)
time_to_predict = datetime(2025, 7, 24, 22, 23, 0) #, tzinfo=timezone.utc)

# Convert the datetime object to Julian Date (jday) and fractional day (fr)
# This is the crucial step for your sgp4 library version
jd, fr = jday(time_to_predict.year, time_to_predict.month,
              time_to_predict.day, time_to_predict.hour,
              time_to_predict.minute, time_to_predict.second + time_to_predict.microsecond / 1e6)

# Propagate the satellite using the two Julian Date arguments
error, r, v = satellite.sgp4(jd, fr) # <-- This is the corrected change!

if error:
    print(f"Error propagating satellite: {error}")
else:
    # --- ECI to ECEF Conversion ---
    # We already have jd and fr, which represent the time,
    # so we can use them for GMST calculation, or use the datetime object.
    # The previous GMST calculation based on datetime is perfectly fine.

    # Calculate Julian Date (JD) and then Greenwich Mean Sidereal Time (GMST)
    # This is a simplified GMST calculation. For high precision, use a dedicated astropy function.
    # Reference: https://en.wikipedia.org/wiki/Sidereal_time#Formulas
    JD_for_GMST = (time_to_predict.toordinal() + 1721424.5 +
                   time_to_predict.hour / 24.0 +
                   time_to_predict.minute / (24.0 * 60.0) +
                   time_to_predict.second / (24.0 * 3600.0) +
                   time_to_predict.microsecond / (24.0 * 3600.0 * 1e6))

    # J2000 epoch (January 1, 2000, 12:00:00 TT) in JD
    JD_2000 = 2451545.0
    T_UT1 = (JD_for_GMST - JD_2000) / 36525.0

    GMST_deg = (280.46061837 + 360.98564736629 * (JD_for_GMST - JD_2000) +
                0.000387933 * T_UT1**2 - T_UT1**3 / 38710000) % 360

    GMST_rad = np.deg2rad(GMST_deg)

    # Rotate ECI vector (r) by GMST to get ECEF vector (r_ecef)
    r_ecef_x = r[0] * np.cos(GMST_rad) + r[1] * np.sin(GMST_rad)
    r_ecef_y = -r[0] * np.sin(GMST_rad) + r[1] * np.cos(GMST_rad)
    r_ecef_z = r[2]
    r_ecef = np.array([r_ecef_x, r_ecef_y, r_ecef_z])

    # --- ECEF to Geodetic (Lat, Lon, Alt) Conversion ---
    a = 6378.137  # WGS84 Earth semi-major axis (km)
    f = 1 / 298.257223563  # WGS84 Earth flattening
    e2 = 2 * f - f**2  # Eccentricity squared

    p = np.sqrt(r_ecef[0]**2 + r_ecef[1]**2)
    lon = np.arctan2(r_ecef[1], r_ecef[0])

    # Initial guess for latitude
    lat = np.arctan2(r_ecef[2], p * (1 - e2))

    # Iterative solution for accurate latitude and altitude
    N = a / np.sqrt(1 - e2 * np.sin(lat)**2)
    alt = p / np.cos(lat) - N

    for _ in range(5):  # Iterate a few times for convergence
        lat_old = lat
        N = a / np.sqrt(1 - e2 * np.sin(lat)**2)
        lat = np.arctan2(r_ecef[2] + N * e2 * np.sin(lat), p)
        if abs(lat - lat_old) < 1e-10:
            break
        alt = p / np.cos(lat) - N

    lon_deg = np.degrees(lon)
    lat_deg = np.degrees(lat)

    # Ensure longitude is in the range [-180, 180]
    if lon_deg > 180:
        lon_deg -= 360
    elif lon_deg < -180:
        lon_deg += 360

    print(f"At {time_to_predict} UTC:")
    print(f"Longitud: {lon_deg:.4f} degrees")
    print(f"Latitud: {lat_deg:.4f} degrees")
    f"Altitud: {alt:.4f} km"

At 2025-07-24 22:23:00 UTC:
Longitud: -28.9089 degrees
Latitud: 24.7133 degrees


In [31]:
error, r, v = satellite.sgp4(jd, fr) # <-- This is the corrected change!
print(error)
print(r)
print(v)

0
(-5032.015872049222, -405.9415546144495, -4748.22777091779)
(-2.418436019341692, -6.470952739222979, 3.118918844365554)


In [24]:
time_to_predict = datetime(2025, 7, 23, 18, 45, 0) #tzinfo=timezone.utc)
print(time_to_predict)

2025-07-23 18:45:00


In [8]:
from sgp4.api import WGS72, Satrec, jday # Import jday
from datetime import datetime, timedelta
#import juliandate as jd
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg

def propagate_tle_orbit(tle_line1, tle_line2, start_time, duration_minutes, step_seconds=60):
    """
    Propaga la órbita de un satélite utilizando TLE.

    Args:
        tle_line1 (str): La primera línea del TLE del satélite.
        tle_line2 (str): La segunda línea del TLE del satélite.
        start_time (datetime): La hora de inicio de la propagación.
        duration_minutes (float): La duración de la propagación en minutos.
        step_seconds (int): El intervalo de tiempo entre cada punto de propagación en segundos.

    Returns:
        list: Una lista de tuplas, donde cada tupla contiene (tiempo, posición_geocentrica, velocidad_geocentrica).
              La posición y velocidad están en coordenadas ECI (Earth-Centered Inertial) en kilómetros y km/s.
    """
    try:
        # Crea un objeto Satrec a partir de las líneas TLE
        satellite = Satrec.twoline2rv(tle_line1, tle_line2)
    except Exception as e:
        print(f"Error al parsear el TLE: {e}")
        return []

    propagated_data = []
    end_time = start_time + timedelta(minutes=duration_minutes)
    current_time = start_time

    while current_time <= end_time:
        # Calcula el tiempo transcurrido desde la época del TLE en minutos
        # sgp4.api.jday convierte una fecha y hora en formato Julian Date y Fractional Day
        jd= satellite.jday
        fr= satellite.deeparg.jfn
        
        # Calcular el delta de tiempo desde la época del TLE en minutos
        # satellite.epoch es un datetime object
        time_since_epoch_minutes = (current_time - satellite.epoch).total_seconds() / 60.0

        # Propaga la órbita
        # r: vector de posición ECI (x, y, z) en km
        # v: vector de velocidad ECI (vx, vy, vz) en km/s
        error, r, v = satellite.sgp4(time_since_epoch_minutes)

        if error == 0:
            propagated_data.append((current_time, r, v))
        else:
            print(f"Advertencia: Error de propagación en el tiempo {current_time}: {error}")

        current_time += timedelta(seconds=step_seconds)

    return propagated_data

if __name__ == "__main__":
    # Ejemplo de uso: TLE del satélite NOAA 15 (puedes reemplazarlo con cualquier otro TLE)
    # TLE obtenido de celestrak.org
    noaa15_tle_line1 = "1 25338U 98030A   25203.12345678  .00000123  00000-0  10000-3 0  9999"
    noaa15_tle_line2 = "2 25338  98.7410 200.0000 0007890 270.0000  90.0000 14.28841000012345"

    # Hora de inicio de la propagación (UTC)
    start_propagation_time = datetime(2025, 7, 24, 0, 0, 0) # 24 de Julio de 2025, 00:00:00 UTC

    # Duración de la propagación (en minutos)
    duration = 120  # 2 horas

    # Intervalo de tiempo para los puntos de propagación (en segundos)
    step = 300 # Cada 5 minutos

    print(f"Propagando órbita para TLE:")
    print(f"Línea 1: {noaa15_tle_line1}")
    print(f"Línea 2: {noaa15_tle_line2}")
    print(f"Desde: {start_propagation_time} (UTC)")
    print(f"Duración: {duration} minutos")
    print(f"Intervalo de paso: {step} segundos\n")

    propagated_points = propagate_tle_orbit(noaa15_tle_line1, noaa15_tle_line2,
                                             start_propagation_time, duration, step)

    if propagated_points:
        print("Puntos de órbita propagados (Tiempo UTC, Posición ECI [km], Velocidad ECI [km/s]):")
        for i, (time, pos, vel) in enumerate(propagated_points):
            if i < 5 or i > len(propagated_points) - 5: # Muestra los primeros 5 y los últimos 5
                print(f"  {time}: Pos={pos}, Vel={vel}")
            elif i == 5 and len(propagated_points) > 10:
                print("  ...")
    else:
        print("No se pudieron propagar puntos de órbita.")

    # Para visualizar estos datos, necesitarías una librería como Matplotlib
    # Por ejemplo, podrías graficar las coordenadas x, y, z en 3D
    # import matplotlib.pyplot as plt
    # from mpl_toolkits.mplot3d import Axes3D
    #
#fig = plt.figure()
#ax = fig.add_subplot(111, projection='3d')
    #
#xs = [p[1][0] for p in propagated_points]
#ys = [p[1][1] for p in propagated_points]
#zs = [p[1][2] for p in propagated_points]
    #
#ax.plot(xs, ys, zs)
#ax.set_xlabel('X (km)')
#ax.set_ylabel('Y (km)')
#ax.set_zlabel('Z (km)')
#ax.set_title('Órbita Propagada en ECI')
#plt.show()

Propagando órbita para TLE:
Línea 1: 1 25338U 98030A   25203.12345678  .00000123  00000-0  10000-3 0  9999
Línea 2: 2 25338  98.7410 200.0000 0007890 270.0000  90.0000 14.28841000012345
Desde: 2025-07-24 00:00:00 (UTC)
Duración: 120 minutos
Intervalo de paso: 300 segundos



AttributeError: 'Satrec' object has no attribute 'jday'

In [85]:
a,b = 2,3

In [86]:
a

2

In [87]:
b

3

In [54]:
from datetime import datetime, timezone
from sgp4.api import WGS72, Satrec, jday # Import jday
import numpy as np
import timeit

def predict_orbit(tle_line1, tle_line2):


    # Create a Satrec object from the TLE
    satellite = Satrec.twoline2rv(tle_line1, tle_line2)

    # Define the time for prediction (UTC)
    time_to_predict = datetime(2025, 7, 25, 11, 19, 0) #, tzinfo=timezone.utc)

    # Convert the datetime object to Julian Date (jday) and fractional day (fr)
    # This is the crucial step for your sgp4 library version
    jd, fr = jday(time_to_predict.year, time_to_predict.month,
                  time_to_predict.day, time_to_predict.hour,
                  time_to_predict.minute, time_to_predict.second + time_to_predict.microsecond / 1e6)

    # Propagate the satellite using the two Julian Date arguments
    error, r, v = satellite.sgp4(jd, fr) # <-- This is the corrected change!

    if error:
        print(f"Error propagating satellite: {error}")
    else:
        # --- ECI to ECEF Conversion ---
        # We already have jd and fr, which represent the time,
        # so we can use them for GMST calculation, or use the datetime object.
        # The previous GMST calculation based on datetime is perfectly fine.

        # Calculate Julian Date (JD) and then Greenwich Mean Sidereal Time (GMST)
        # This is a simplified GMST calculation. For high precision, use a dedicated astropy function.
        # Reference: https://en.wikipedia.org/wiki/Sidereal_time#Formulas
        JD_for_GMST = (time_to_predict.toordinal() + 1721424.5 +
                       time_to_predict.hour / 24.0 +
                       time_to_predict.minute / (24.0 * 60.0) +
                       time_to_predict.second / (24.0 * 3600.0) +
                       time_to_predict.microsecond / (24.0 * 3600.0 * 1e6))

        # J2000 epoch (January 1, 2000, 12:00:00 TT) in JD
        JD_2000 = 2451545.0
        T_UT1 = (JD_for_GMST - JD_2000) / 36525.0

        GMST_deg = (280.46061837 + 360.98564736629 * (JD_for_GMST - JD_2000) +
                    0.000387933 * T_UT1**2 - T_UT1**3 / 38710000) % 360

        GMST_rad = np.deg2rad(GMST_deg)

        # Rotate ECI vector (r) by GMST to get ECEF vector (r_ecef)
        r_ecef_x = r[0] * np.cos(GMST_rad) + r[1] * np.sin(GMST_rad)
        r_ecef_y = -r[0] * np.sin(GMST_rad) + r[1] * np.cos(GMST_rad)
        r_ecef_z = r[2]
        r_ecef = np.array([r_ecef_x, r_ecef_y, r_ecef_z])

        # --- ECEF to Geodetic (Lat, Lon, Alt) Conversion ---
        a = 6378.137  # WGS84 Earth semi-major axis (km)
        f = 1 / 298.257223563  # WGS84 Earth flattening
        e2 = 2 * f - f**2  # Eccentricity squared

        p = np.sqrt(r_ecef[0]**2 + r_ecef[1]**2)
        lon = np.arctan2(r_ecef[1], r_ecef[0])

        # Initial guess for latitude
        lat = np.arctan2(r_ecef[2], p * (1 - e2))

        # Iterative solution for accurate latitude and altitude
        N = a / np.sqrt(1 - e2 * np.sin(lat)**2)
        alt = p / np.cos(lat) - N

        for _ in range(5):  # Iterate a few times for convergence
            lat_old = lat
            N = a / np.sqrt(1 - e2 * np.sin(lat)**2)
            lat = np.arctan2(r_ecef[2] + N * e2 * np.sin(lat), p)
            if abs(lat - lat_old) < 1e-10:
                break
            alt = p / np.cos(lat) - N

        lon_deg = np.degrees(lon)
        lat_deg = np.degrees(lat)

        # Ensure longitude is in the range [-180, 180]
        if lon_deg > 180:
            lon_deg -= 360
        elif lon_deg < -180:
            lon_deg += 360

        print(f"At {time_to_predict} UTC:")
        print(f"Longitud: {lon_deg:.4f} degrees")
        print(f"Latitud: {lat_deg:.4f} degrees")
        f"Altitud: {alt:.4f} km"




In [55]:
import timeit
# Your provided TLEs
tle_line1 = "1 44726U 19074P   25205.00002315  .00150861  00000-0  12939-2 0  9996"
tle_line2 = "2 44726  53.0486 106.7361 0009585  76.9653 162.6220 15.68376899  5914"



predict_orbit(tle_line1, tle_line2)

At 2025-07-25 11:19:00 UTC:
Longitud: -97.1802 degrees
Latitud: -53.0244 degrees


In [89]:
from datetime import datetime, timedelta, timezone

current_time_utc= datetime.now(timezone.utc)
print(current_time_utc)

for hour_to_add in range(745):
    #print(hour_to_add)
    time_increment= timedelta(hours=hour_to_add)
    future_time = current_time_utc + time_increment
    #print(type(future_time))
    
    year=future_time.year
    month=future_time.month
    day=future_time.day
    hour=future_time.hour

    print(f"{year}- {month}- {day}- {hour}")



# Using f-strings (formatted string literals) for cleaner concatenation
    #name = "Alice"
    #age = 30
    #formatted_string = f"My name is {name} and I am {age} years old."
    #print(formatted_string)
#print(year)
#print(month)
#print(day)
#print(hour)
    

2025-07-25 22:15:19.943922+00:00
2025- 7- 25- 22
2025- 7- 25- 23
2025- 7- 26- 0
2025- 7- 26- 1
2025- 7- 26- 2
2025- 7- 26- 3
2025- 7- 26- 4
2025- 7- 26- 5
2025- 7- 26- 6
2025- 7- 26- 7
2025- 7- 26- 8
2025- 7- 26- 9
2025- 7- 26- 10
2025- 7- 26- 11
2025- 7- 26- 12
2025- 7- 26- 13
2025- 7- 26- 14
2025- 7- 26- 15
2025- 7- 26- 16
2025- 7- 26- 17
2025- 7- 26- 18
2025- 7- 26- 19
2025- 7- 26- 20
2025- 7- 26- 21
2025- 7- 26- 22
2025- 7- 26- 23
2025- 7- 27- 0
2025- 7- 27- 1
2025- 7- 27- 2
2025- 7- 27- 3
2025- 7- 27- 4
2025- 7- 27- 5
2025- 7- 27- 6
2025- 7- 27- 7
2025- 7- 27- 8
2025- 7- 27- 9
2025- 7- 27- 10
2025- 7- 27- 11
2025- 7- 27- 12
2025- 7- 27- 13
2025- 7- 27- 14
2025- 7- 27- 15
2025- 7- 27- 16
2025- 7- 27- 17
2025- 7- 27- 18
2025- 7- 27- 19
2025- 7- 27- 20
2025- 7- 27- 21
2025- 7- 27- 22
2025- 7- 27- 23
2025- 7- 28- 0
2025- 7- 28- 1
2025- 7- 28- 2
2025- 7- 28- 3
2025- 7- 28- 4
2025- 7- 28- 5
2025- 7- 28- 6
2025- 7- 28- 7
2025- 7- 28- 8
2025- 7- 28- 9
2025- 7- 28- 10
2025- 7- 28- 11
2025-

In [71]:
from datetime import datetime, timedelta

current_time = datetime.now()
print(current_time)

# Define the number of hours to add
hours_to_add = 744

# Create a timedelta object representing the duration
time_increment = timedelta(hours=hours_to_add)

# Add the timedelta to the current datetime
future_time = current_time + time_increment

# Print the original and new datetimes
print(f"Current time: {current_time}")
print(f"Future time: {future_time}")

2025-07-25 12:11:52.333178
Current time: 2025-07-25 12:11:52.333178
Future time: 2025-08-25 12:11:52.333178


In [1]:
tle_line1 = "1 23168U 94038A   25205.46876539 -.00000083  00000-0  00000-0 0  9998"

In [4]:
tle_line1[2:-62]

'23168'

In [ ]:
from datetime import datetime, timezone, timedelta
from sgp4.api import WGS72, Satrec, jday # Import jday
import numpy as np
import time


def predict_orbit(tle_line1, tle_line2, year, month, day, hour):


    # Create a Satrec object from the TLE
    satellite = Satrec.twoline2rv(tle_line1, tle_line2)

    # Define the time for prediction (UTC)
    time_to_predict = datetime(year, month, day, hour, 0, 0) #, tzinfo=timezone.utc)

    # Convert the datetime object to Julian Date (jday) and fractional day (fr)
    # This is the crucial step for your sgp4 library version
    jd, fr = jday(time_to_predict.year, time_to_predict.month,
                  time_to_predict.day, time_to_predict.hour,
                  time_to_predict.minute, time_to_predict.second + time_to_predict.microsecond / 1e6)

    # Propagate the satellite using the two Julian Date arguments
    error, r, v = satellite.sgp4(jd, fr) # <-- This is the corrected change!

    if error:
        print(f"Error propagating satellite: {error}")
    else:
        # --- ECI to ECEF Conversion ---
        # We already have jd and fr, which represent the time,
        # so we can use them for GMST calculation, or use the datetime object.
        # The previous GMST calculation based on datetime is perfectly fine.

        # Calculate Julian Date (JD) and then Greenwich Mean Sidereal Time (GMST)
        # This is a simplified GMST calculation. For high precision, use a dedicated astropy function.
        # Reference: https://en.wikipedia.org/wiki/Sidereal_time#Formulas
        JD_for_GMST = (time_to_predict.toordinal() + 1721424.5 +
                       time_to_predict.hour / 24.0 +
                       time_to_predict.minute / (24.0 * 60.0) +
                       time_to_predict.second / (24.0 * 3600.0) +
                       time_to_predict.microsecond / (24.0 * 3600.0 * 1e6))

        # J2000 epoch (January 1, 2000, 12:00:00 TT) in JD
        JD_2000 = 2451545.0
        T_UT1 = (JD_for_GMST - JD_2000) / 36525.0

        GMST_deg = (280.46061837 + 360.98564736629 * (JD_for_GMST - JD_2000) +
                    0.000387933 * T_UT1**2 - T_UT1**3 / 38710000) % 360

        GMST_rad = np.deg2rad(GMST_deg)

        # Rotate ECI vector (r) by GMST to get ECEF vector (r_ecef)
        r_ecef_x = r[0] * np.cos(GMST_rad) + r[1] * np.sin(GMST_rad)
        r_ecef_y = -r[0] * np.sin(GMST_rad) + r[1] * np.cos(GMST_rad)
        r_ecef_z = r[2]
        r_ecef = np.array([r_ecef_x, r_ecef_y, r_ecef_z])

        # --- ECEF to Geodetic (Lat, Lon, Alt) Conversion ---
        a = 6378.137  # WGS84 Earth semi-major axis (km)
        f = 1 / 298.257223563  # WGS84 Earth flattening
        e2 = 2 * f - f**2  # Eccentricity squared

        p = np.sqrt(r_ecef[0]**2 + r_ecef[1]**2)
        lon = np.arctan2(r_ecef[1], r_ecef[0])

        # Initial guess for latitude
        lat = np.arctan2(r_ecef[2], p * (1 - e2))

        # Iterative solution for accurate latitude and altitude
        N = a / np.sqrt(1 - e2 * np.sin(lat)**2)
        alt = p / np.cos(lat) - N

        for _ in range(5):  # Iterate a few times for convergence
            lat_old = lat
            N = a / np.sqrt(1 - e2 * np.sin(lat)**2)
            lat = np.arctan2(r_ecef[2] + N * e2 * np.sin(lat), p)
            if abs(lat - lat_old) < 1e-10:
                break
            alt = p / np.cos(lat) - N

        lon_deg = np.degrees(lon)
        lat_deg = np.degrees(lat)

        # Ensure longitude is in the range [-180, 180]
        if lon_deg > 180:
            lon_deg -= 360
        elif lon_deg < -180:
            lon_deg += 360

        print(f"At {time_to_predict} UTC:")
        print(f"Longitud: {lon_deg:.4f} degrees")
        print(f"Latitud: {lat_deg:.4f} degrees")
        f"Altitud: {alt:.4f} km"

        # CONDICION PARA APENDEAR 
        #if lon_deg ...

        write= tle_line1[2:-62]+"\nLongitud: "+str(lon_deg)+"\nLatitud: "+str(lat_deg)+"\n\n"
        print(write)
        #textfile= open("C:\\Users\\Joel\\Documents\\Python\\email\\Perigee_Apogee_Range_v2.txt", "a")
        #textfile.write(write+"\n\n")
        #textfile.close()
            


#TLEs
tle_line1 = "1 23168U 94038A   25210.45598799 -.00000063  00000-0  00000+0 0  9999"
tle_line2 = "2 23168  13.9250 355.0213 0007910 130.2201 235.0716  1.00162738113767"


#Increment in date
current_time_utc= datetime.now(timezone.utc)
#print(current_time)

#start_time = time.time()  # Record the start time
for hour_to_add in range(745):
    #print(hour_to_add)
    time_increment= timedelta(hours=hour_to_add)
    future_time = current_time_utc + time_increment
    #print(type(future_time))
    
    year=future_time.year
    month=future_time.month
    day=future_time.day
    hour=future_time.hour

    predict_orbit(tle_line1, tle_line2, year, month, day, hour)
    #print(f"{year}- {month}- {day}- {hour}")
#end_time = time.time()    # Record the end time
#execution_time = end_time - start_time
#print(f"Execution time: {execution_time:.4f} seconds")






# CODE TO TEST THE TIME OF EXECUTION OF A FUNCTION

#start_time = time.time()  # Record the start time
# Your code goes here

#end_time = time.time()    # Record the end time
#execution_time = end_time - start_time
#print(f"Execution time: {execution_time:.4f} seconds")
